# Importing Libraries

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import os

In [2]:
path = r'C:\Users\PC\Downloads\MM-2023 Instacart Basket Analysis'

In [5]:
df = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_fixed.pkl'))

## Step 2 & 3

In [7]:
df_sub = df[:1000000]

In [10]:
# Finding aggregated mean of the “order_number” column grouped by “department_id” of subset
df_sub.groupby('department_id').agg({'order_number': ['mean']})

,order_number
,mean
department_id,
4,18.825780
7,17.472355
13,17.993423
14,19.246334
16,19.463012
17,11.294069
19,19.305237
20,17.599636


In [11]:
# Finding aggregated mean of the “order_number” column grouped by “department_id” of entire dataframe
df.groupby('department_id').agg({'order_number': ['mean']})

,order_number
,mean
department_id,
1,15.457838
2,17.277920
3,17.170395
4,17.811403
5,15.215751
6,16.439806
7,17.225802
8,15.340650


The main difference is that the subset does not include all departments. The numbers of the departments we do have are also slightly different.

## Step 4

In [14]:
# Creating max order column
df['max_order'] = df.groupby(['user_id'])['order_number'].transform(np.max)

In [15]:
# Creating loyalty flag
df.loc[df['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'
df.loc[(df['max_order'] <= 40) & (df['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'
df.loc[df['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [16]:
df['loyalty_flag'].value_counts(dropna = False)

Regular customer    15876776
Loyal customer      10284093
New customer         6243990
Name: loyalty_flag, dtype: int64

## Step 5

In [17]:
# Checking mean, min, max and sum of different loyalty tiers by spending
df.groupby('loyalty_flag').agg({'prices': ['mean', 'min', 'max', 'sum']})

prices                           
                       mean  min      max          sum
loyalty_flag                                          
Loyal customer    10.386336  1.0  99999.0  106814042.2
New customer      13.294670  1.0  99999.0   83011787.2
Regular customer  12.495717  1.0  99999.0  198391693.2

## Step 6

In [18]:
# Adding average_price column
df['average_price'] = df.groupby(['user_id'])['prices'].transform(np.mean)

In [20]:
# Creating spending flag
df.loc[df['average_price'] < 10, 'spending_flag'] = 'Low spender'
df.loc[df['average_price'] >= 10, 'spending_flag'] = 'High spender'

In [21]:
df['spending_flag'].value_counts(dropna = False)

Low spender     31770614
High spender      634245
Name: spending_flag, dtype: int64

## Step 7

In [24]:
# Adding average_dspo column
df['average_dspo'] = df.groupby(['user_id'])['days_since_prior_order'].transform(np.mean)

In [25]:
# Creating frequency flag
df.loc[df['average_dspo'] > 20, 'frequency_flag'] = 'Non-frequent customer'
df.loc[(df['average_dspo'] <= 20) & (df['average_dspo'] > 10), 'frequency_flag'] = 'Regular customer'
df.loc[df['average_dspo'] <= 10, 'frequency_flag'] = 'Frequent customer'

In [26]:
df['frequency_flag'].value_counts(dropna = False)

Frequent customer        16468582
Regular customer         12521159
Non-frequent customer     3415113
NaN                             5
Name: frequency_flag, dtype: int64

## Step 9

In [27]:
df.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'orders_products_flags.pkl'))